In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:


# standings_url = 'https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats'

# data = requests.get(standings_url)

# soup = BeautifulSoup(data.text)
# standings_table= soup.select('table.stats_table')[0]
# links = standings_table.find_all('a')

# links = [l.get('href') for l in links]

# links = [l for l in links if '/squads/' in l ]

# team_urls = [f'https://fbref.com{l}' for l in links]
# team_url = team_urls[0]
# data= requests.get(team_url)
# import pandas as pd

# matches = pd.read_html(data.text, match='Scores & Fixtures')
# soup = BeautifulSoup(data.text)
# links = soup.find_all('a')

# links = [l.get('href') for l in links]
# links = [l for l in links if l and 'all_comps/shooting/' in l]

# data = requests.get(f'https://fbref.com{links[0]}')
# shooting = pd.read_html(data.text,match='Shooting')[0]
# shooting.columns = shooting.columns.droplevel()
# shooting.columns
# team_data = matches[0].merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK',
#        'PK', 'PKatt', 'xG'
#        ]],on='Date')


In [6]:


lista_ligas = ['https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats',
               "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats",
               "https://fbref.com/en/comps/11/2021-2022/2021-2022-Serie-A-Stats",
               "https://fbref.com/en/comps/13/2021-2022/2021-2022-Ligue-1-Stats",
               "https://fbref.com/en/comps/20/2021-2022/2021-2022-Bundesliga-Stats",
               ]

years = list(range(2021, 2015, -1))
all_matches = []
for year in years:
  for liga in lista_ligas:
    data = requests.get(liga)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]

        links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
        data_shooting = requests.get(f"https://fbref.com{links_shooting[0]}")

        links_passing = [l for l in links if l and 'all_comps/passing/' in l]
        data_passing = requests.get(f"https://fbref.com{links_passing[0]}")

        links_defensive = [l for l in links if l and 'all_comps/defense/' in l]
        data_defensive = requests.get(f"https://fbref.com{links_defensive[0]}")

        shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
        passing = pd.read_html(data_passing.text, match="Passing")[0]
        defensive = pd.read_html(data_defensive.text, match="Defensive Actions")[0]

        shooting.columns = shooting.columns.droplevel()
        passing.columns = passing.columns.droplevel()
        defensive.columns = defensive.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt",'xG']], on="Date")
        except ValueError:
            continue
       

        try:
            team_data = matches.merge(passing[["Date",'Cmp','Att','TotDist','PrgDist','Ast','xAG', 'xA','KP']], on="Date")
        except ValueError:
            continue
        
		
        try:
            team_data = matches.merge(defensive[["Date",'Tkl',	'TklW'	,'Def 3rd'	,'Mid 3rd',	'Att 3rd','Tkl'	,'Att',	'Tkl%'	,'Lost'	,'Blocks',	'Sh',	'Pass'	,'Int'	,'Tkl+Int'	,'Clr',	'Err' ]], on="Date")
        except ValueError:
            continue


          
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(5)

     

   

   

ValueError: ignored

In [7]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]


In [ ]:
match_df

In [9]:
match_df.to_csv('matches.csv')